### Testing for parsing some KPIs from log file.

In [2]:
import pandas as pd
import re
import datetime

def extract_box_name(line):
    match = re.findall(r'([A-Z]+_\d+)', str(line))
    if match:
        return match[0]
    else:
        return None

def extract_line(line):
    match = re.findall(r'([A-Z]+_\d+)_(.+)', str(line))
    if match:
        return match[0][1]
    else:
        return None

# read data
df = pd.read_excel('SE_Onlinelog_230810.xlsm', sheet_name='Survey', skiprows=2)

# handle date and time
df['Time'] = df['Time'].astype(str).str.split('.').str[0]
df['Date_Time'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time'])

# extract box_name and line from "Run Line/Target" using regex
df['box_name'] = df['Run Line/Target'].apply(extract_box_name)
df['line'] = df['Run Line/Target'].apply(extract_line)

# filter Log on and Log off events
df = df[df['Event'].isin(['Log on', 'Log off', 'Log on DNP'])]

# sort the values
df = df.sort_values(['box_name', 'line', 'Date_Time'])

# list to store rows
rows = []

# iterate over the dataframe
for box_name in df['box_name'].dropna().unique():
    temp_df = df[df['box_name'] == box_name]
    for line in temp_df['line'].dropna().unique():
        line_df = temp_df[temp_df['line'] == line]
        
        total_duration_seconds = 0
        start_time = None
        multiple_runs = 0
        for i, row in line_df.iterrows():
            if row['Event'] in ['Log on', 'Log on DNP']:
                start_time = row['Date_Time']
                multiple_runs += 1
            elif row['Event'] == 'Log off' and start_time is not None:
                duration_seconds = int((row['Date_Time'] - start_time).total_seconds())
                total_duration_seconds += duration_seconds
                start_time = None

        if total_duration_seconds > 0:
            duration = str(datetime.timedelta(seconds=total_duration_seconds))

            multiple_runs_text = f"Multiple runs: {multiple_runs}" if multiple_runs > 1 else ""
            row = {
                'box_name': box_name,
                'line': line,
                'operator': line_df.iloc[0]['Op.'],
                'start_time': line_df.iloc[0]['Date_Time'].strftime('%Y-%m-%d %H:%M:%S'),
                'end_time': line_df.iloc[-1]['Date_Time'].strftime('%Y-%m-%d %H:%M:%S'),
                'duration': duration,
                'multiple_runs': multiple_runs_text
            }
            rows.append(row)

# create dataframe from rows
result = pd.DataFrame(rows)

# save to excel
result.to_excel('output.xlsx', index=False)


## Adding Lenght Column

In [3]:
import pandas as pd
import re
import datetime
from geopy.distance import geodesic

def extract_box_name(line):
    match = re.findall(r'([A-Z]+_\d+)', str(line))
    if match:
        return match[0]
    else:
        return None

def extract_line(line):
    match = re.findall(r'([A-Z]+_\d+)_(.+)', str(line))
    if match:
        return match[0][1]
    else:
        return None

def parse_coords(coord_str):
    if pd.isnull(coord_str):
        return None
    deg, minutes, seconds, direction = re.split('[^\d\.]', coord_str)[:4]
    decimal_deg = float(deg) + float(minutes)/60 + float(seconds)/(60*60)
    if direction in ['S', 'W']:
        decimal_deg *= -1
    return decimal_deg

def calculate_distance(start_coords, end_coords):
    return geodesic(start_coords, end_coords).meters

df = pd.read_excel('Onlinelog.xlsm', sheet_name='Survey', skiprows=2)

df['Time'] = df['Time'].astype(str).str.split('.').str[0]
df['Date_Time'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time'])

df['box_name'] = df['Run Line/Target'].apply(extract_box_name)
df['line'] = df['Run Line/Target'].apply(extract_line)

df = df[df['Event'].str.lower().str.startswith(('log on', 'log off'))]

df = df.sort_values(['box_name', 'line', 'Date_Time'])

rows = []

for box_name in df['box_name'].dropna().unique():
    temp_df = df[df['box_name'] == box_name]
    for line in temp_df['line'].dropna().unique():
        line_df = temp_df[temp_df['line'] == line]
        
        total_duration_seconds = 0
        total_distance = 0
        start_time = None
        start_coords = None
        multiple_runs = 0
        for i, row in line_df.iterrows():
            event_lower = row['Event'].lower()
            if event_lower.startswith('log on'):
                start_time = row['Date_Time']
                lat = parse_coords(row['SE Lat'])
                long = parse_coords(row['SE Long'])
                if lat is not None and long is not None:
                    start_coords = (lat, long)
                multiple_runs += 1
            elif event_lower.startswith('log off') and start_time is not None and start_coords is not None:
                duration_seconds = int((row['Date_Time'] - start_time).total_seconds())
                total_duration_seconds += duration_seconds
                start_time = None

                lat = parse_coords(row['SE Lat'])
                long = parse_coords(row['SE Long'])
                if lat is not None and long is not None:
                    end_coords = (lat, long)
                    distance = calculate_distance(start_coords, end_coords)
                    total_distance += distance
                start_coords = None

        if total_duration_seconds > 0:
            duration = str(datetime.timedelta(seconds=total_duration_seconds))
            distance = f"{total_distance / 1000:.2f} km"
            row = {
                'box_name': box_name,
                'line': line,
                'operator': line_df.iloc[0]['Op.'],
                'from': line_df['Date_Time'].min().strftime("%Y-%m-%d %H:%M:%S"),
                'to': line_df['Date_Time'].max().strftime("%Y-%m-%d %H:%M:%S"),
                'duration': duration,
                'distance': distance,
                'runs': multiple_runs
            }
            rows.append(row)

df_output = pd.DataFrame(rows)
df_output.to_excel('output.xlsx', index=False)


### More columns with info

In [4]:
import pandas as pd
import re
import datetime
from geopy.distance import geodesic

def extract_box_name(line):
    match = re.findall(r'([A-Z]+_\d+)', str(line))
    if match:
        return match[0]
    else:
        return None

def extract_line(line):
    match = re.findall(r'([A-Z]+_\d+)_(.+)', str(line))
    if match:
        return match[0][1]
    else:
        return None

def parse_coords(coord_str):
    if pd.isnull(coord_str):
        return None
    deg, minutes, seconds, direction = re.split('[^\d\.]', coord_str)[:4]
    decimal_deg = float(deg) + float(minutes)/60 + float(seconds)/(60*60)
    if direction in ['S', 'W']:
        decimal_deg *= -1
    return decimal_deg

# read data
df = pd.read_excel('SE_Onlinelog_230810.xlsm', sheet_name='Survey', skiprows=2)

# handle date and time
df['Time'] = df['Time'].astype(str).str.split('.').str[0]
df['Date_Time'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time'])

# extract box_name and line from "Run Line/Target" using regex
df['box_name'] = df['Run Line/Target'].apply(extract_box_name)
df['line'] = df['Run Line/Target'].apply(extract_line)

# convert coords to decimal
df['SE Lat'] = df['SE Lat'].apply(parse_coords)
df['SE Long'] = df['SE Long'].apply(parse_coords)

# filter Log on and Log off events
df = df[df['Event'].str.lower().str.contains('log on') | df['Event'].str.lower().str.contains('log off')]

# sort the values
df = df.sort_values(['box_name', 'line', 'Date_Time'])

# list to store rows
rows = []

# iterate over the dataframe
for box_name in df['box_name'].dropna().unique():
    temp_df = df[df['box_name'] == box_name]
    for line in temp_df['line'].dropna().unique():
        line_df = temp_df[temp_df['line'] == line]

        total_duration_seconds = 0
        total_distance = 0
        start_time = None
        start_coords = None
        multiple_runs = 0
        for i, row in line_df.iterrows():
            if 'log on' in row['Event'].lower():
                start_time = row['Date_Time']
                start_coords = (row['SE Lat'], row['SE Long'])
                multiple_runs += 1
            elif 'log off' in row['Event'].lower() and start_time is not None:
                duration_seconds = int((row['Date_Time'] - start_time).total_seconds())
                total_duration_seconds += duration_seconds
                if start_coords and not any(pd.isnull(coord) for coord in start_coords) and not any(pd.isnull(coord) for coord in (row['SE Lat'], row['SE Long'])):
                    distance = geodesic(start_coords, (row['SE Lat'], row['SE Long'])).km
                    total_distance += distance
                start_time = None
                start_coords = None

        if total_duration_seconds > 0:
            duration = str(datetime.timedelta(seconds=total_duration_seconds))

            row = {
                'box_name': box_name,
                'line': line,
                'operator': line_df.iloc[0]['Op.'],
                'from': line_df['Date_Time'].min().strftime("%d/%m/%Y %H:%M:%S"),
                'to': line_df['Date_Time'].max().strftime("%d/%m/%Y %H:%M:%S"),
                'duration': duration,
                'distance': f'{total_distance:.2f} km',
                'runs': multiple_runs,
                'Area': line_df.iloc[0]['Area'],
                'Qinsy DB': line_df.iloc[0]['Qinsy DB'],
                'NaviScan': line_df.iloc[0]['NaviScan'],
                'SSS': line_df.iloc[0]['SSS'],
                'Sprint': line_df.iloc[0]['Sprint'],
            }
            rows.append(row)

# create a dataframe from the list of rows
df_output = pd.DataFrame(rows)
df_output.to_excel('output.xlsx', index=False)


### Analytics. New sheet

In [6]:
import pandas as pd

# load the data
df = pd.read_excel('output.xlsx')

# Convert 'from' and 'to' columns to datetime format
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Create new column 'duration_minutes' to hold the duration in minutes
df['duration_minutes'] = (df['to'] - df['from']).dt.total_seconds() / 60

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Teams Efficiency: Total distance covered and total time spent by each teams
teams_efficiency = df.groupby('teams').agg({'distance': 'sum', 'duration_minutes': 'sum'})
teams_efficiency['efficiency'] = teams_efficiency['distance'] / teams_efficiency['duration_minutes']

# Average Duration per Line in Box
average_duration_per_line_in_box = df.groupby('box_name')['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

# Average Distance Covered per Box
average_distance_per_box = df.groupby('box_name')['distance'].mean()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR) per Box
retries_per_box = df[df['line'].str.contains('RR')]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

descriptions = {
    'Teams Efficiency': 'Total distance covered and total time spent by each teams. Efficiency is calculated as total distance divided by total time.',
    'Average Duration per Line in Box': 'The average duration of each line survey conducted for each box, measured in minutes.',
    'Total Duration per Box': 'The total duration of all surveys conducted for each box, measured in minutes.',
    'Average Distance per Box': 'The average distance covered during surveys of each box, measured in kilometers.',
    'Survey Frequency per Box': 'The number of surveys conducted for each box.',
    'Retries per Box': 'The number of retries (RR) conducted for each box.',
}

# Initialize ExcelWriter to write to an Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    pd.DataFrame.from_dict(descriptions, orient='index', columns=['Description']).to_excel(writer, sheet_name='Description')
    teams_efficiency.to_excel(writer, sheet_name='Teams Efficiency')
    average_duration_per_line_in_box.to_excel(writer, sheet_name='Average Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    average_distance_per_box.to_excel(writer, sheet_name='Average Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')

print("Analysis complete. Results stored in 'marine_survey_analysis.xlsx'.")


Analysis complete. Results stored in 'marine_survey_analysis.xlsx'.


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\461657533.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\461657533.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
c:\Users\Edin.Smlatic\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [7]:
##################### NEW TRY ################################


In [8]:
import pandas as pd

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

# load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)

# Total Lines Completed by Each Team
total_lines_by_team = df.groupby('teams')['line'].count()

# Total Boxes Completed by Each Team
total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

# Combine these metrics into a DataFrame
team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

# Total Distance Covered per Box
total_distance_per_box = df.groupby('box_name')['distance'].sum()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR or INF or in) per Box
retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

# Duration per Line in Box
duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

descriptions = {
    'Teams': 'The total amount of lines and boxes completed by each team.',
    'Total Distance per Box': 'The total distance covered during surveys of each box, measured in kilometers.',
    'Survey Frequency per Box': 'The number of times each box has been surveyed. It is calculated by counting the occurrences of each box name in the data.',
    'Retries per Box': 'The number of retries (RR, INF, in) conducted for each box.',
    'Duration per Line in Box': 'The duration of each line survey conducted for each box, measured in minutes.',
    'Total Duration per Box': 'The total duration of all surveys conducted for each box, measured in minutes.'
}

# Initialize ExcelWriter to write to an Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    pd.DataFrame.from_dict(descriptions, orient='index', columns=['Description']).to_excel(writer, sheet_name='Description')
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\121268448.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)


### Adding time spent traveling bewteen boxes.

In [9]:
import pandas as pd

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

# load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            # Find the minimum "from" timestamp that is after the "last_log_off" timestamp
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  # Skip if there are no "from" timestamps after "last_log_off"
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
        
    last_log_off = row['to']  # update log off time with each iteration, it will become the latest "to" timestamp of this box

# Add a 'day' column for grouping
df['day'] = df['from'].dt.date

# Group by both box_name and day, then get the min 'from' and max 'to' for each group
grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

# Calculate the duration for each group, convert to minutes
grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

# Sum up the durations for each box across all days
total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

# Convert 'from' and 'to' columns back into string format
df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Metrics computation continues as before...

# Total Lines Completed by Each Team
total_lines_by_team = df.groupby('teams')['line'].count()

# Total Boxes Completed by Each Team
total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

# Combine these metrics into a DataFrame
team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

# Total Distance Covered per Box
total_distance_per_box = df.groupby('box_name')['distance'].sum()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR or INF or in) per Box
retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

# Duration per Line in Box
duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

descriptions = {
    'Teams': 'The total amount of lines and boxes completed by each team.',
    'Total Distance per Box': 'The total distance covered during surveys of each box, measured in kilometers.',
    'Survey Frequency per Box': 'The number of times each box has been surveyed. It is calculated by counting the occurrences of each box name in the data.',
    'Retries per Box': 'The number of retries (RR, INF, in) conducted for each box.',
    'Duration per Line in Box': 'The duration of each line survey conducted for each box, measured in minutes.',
    'Total Duration per Box': 'The total duration of all surveys conducted for each box, measured in minutes.',
    'Log Off to Log On': 'The time difference in minutes between logging off from one box and logging on to the next.',
    'Total Duration per Box with Relocation': 'The total duration of all surveys conducted for each box including relocation times, measured in minutes.'
}

# Initialize ExcelWriter to write to an Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    pd.DataFrame.from_dict(descriptions, orient='index', columns=['Description']).to_excel(writer, sheet_name='Description')
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Log Off to Log On', index=False)  # new dataframe
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation', index=True)  # new dataframe


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1580225504.py:21: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1580225504.py:22: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1580225504.py:87: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
c:\Users\Edin.Smlatic\AppData\Local\Programs\Python\Python311\Lib\site-pa

### Completed Lines Sheet

In [10]:
import pandas as pd

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df

# load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            # Find the minimum "from" timestamp that is after the "last_log_off" timestamp
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  # Skip if there are no "from" timestamps after "last_log_off"
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
        
    last_log_off = row['to']  # update log off time with each iteration, it will become the latest "to" timestamp of this box

# Add a 'day' column for grouping
df['day'] = df['from'].dt.date

# Group by both box_name and day, then get the min 'from' and max 'to' for each group
grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

# Calculate the duration for each group, convert to minutes
grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

# Sum up the durations for each box across all days
total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

# Convert 'from' and 'to' columns back into string format
df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Metrics computation continues as before...

# Total Lines Completed by Each Team
total_lines_by_team = df.groupby('teams')['line'].count()

# Total Boxes Completed by Each Team
total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

# Combine these metrics into a DataFrame
team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

# Total Distance Covered per Box
total_distance_per_box = df.groupby('box_name')['distance'].sum()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR or INF or in) per Box
retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

# Duration per Line in Box
duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

# Read and process both text files with the completed tasks info
completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

# Concatenate the dataframes
df_combined = pd.concat([df1, df2])

# Initialize ExcelWriter to write to an Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Log Off to Log On', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation')
    df_combined.to_excel(writer, sheet_name='Progress', index=False)


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\487925414.py:42: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\487925414.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\487925414.py:108: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
c:\Users\Edin.Smlatic\AppData\Local\Programs\Python\Python311\Lib\site-pack

### Adding Summary tab

In [12]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df

# Load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            # Find the minimum "from" timestamp that is after the "last_log_off" timestamp
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  # Skip if there are no "from" timestamps after "last_log_off"
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
        
    last_log_off = row['to']  # update log off time with each iteration, it will become the latest "to" timestamp of this box

# Add a 'Apologies for the abrupt cut-off. Here is the remainder of the script:


# Add a 'day' column for grouping
df['day'] = df['from'].dt.date

# Group by both box_name and day, then get the min 'from' and max 'to' for each group
grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

# Calculate the duration for each group, convert to minutes
grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

# Sum up the durations for each box across all days
total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

# Convert 'from' and 'to' columns back into string format
df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Compute the total distance surveyed
total_distance_surveyed = df['distance'].sum()

# Total Lines Completed by Each Team
total_lines_by_team = df.groupby('teams')['line'].count()

# Total Boxes Completed by Each Team
total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

# Combine these metrics into a DataFrame
team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

# Total Distance Covered per Box
total_distance_per_box = df.groupby('box_name')['distance'].sum()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR or INF or in) per Box
retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

# Duration per Line in Box
duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

# Read and process both text files with the completed tasks info
completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

# Concatenate the dataframes
df_combined = pd.concat([df1, df2])

# Compute the metrics for the Summary sheet
completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

avg_time_between_boxes = box_changes['time_diff'].mean()
bad_weather_wait = box_changes[box_changes['time_diff'] > 1000]['time_diff'].mean()

# Assuming "BH_" and "CPT_" in box names indicate the type of the boxes
median_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].median()
median_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].median()

#```python
# Create a summary DataFrame
summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage,  total_distance_surveyed, median_BH_box_time, median_CPT_box_time, avg_time_between_boxes, bad_weather_wait]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Median BH Box Time (Minutes)', 'Median CPT Box Time (Minutes)', 'Average Time Between Boxes (Minutes)', 'Average Downtime due to Weather (Minutes)']
)

# Transpose the summary DataFrame
summary_df = summary_df.T

# Initialize ExcelWriter to write to an Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation')
    df_combined.to_excel(writer, sheet_name='Progress', index=False)


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3085874899.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3085874899.py:44: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3085874899.py:113: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3085874899.py:13

### Adding date log

In [13]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df

# Load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            # Find the minimum "from" timestamp that is after the "last_log_off" timestamp
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  # Skip if there are no "from" timestamps after "last_log_off"
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
        
    last_log_off = row['to']  # update log off time with each iteration, it will become the latest "to" timestamp of this box

# Add a 'day' column for grouping
df['day'] = df['from'].dt.date

# Group by both box_name and day, then get the min 'from' and max 'to' for each group
grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

# Calculate the duration for each group, convert to minutes
grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

# Sum up the durations for each box across all days
total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

# Convert 'from' and 'to' columns back into string format
df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Compute the total distance surveyed
total_distance_surveyed = df['distance'].sum()

# Total Lines Completed by Each Team
total_lines_by_team = df.groupby('teams')['line'].count()

# Total Boxes Completed by Each Team
total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

# Combine these metrics into a DataFrame
team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

# Total Distance Covered per Box
total_distance_per_box = df.groupby('box_name')['distance'].sum()

# Survey Frequency per Box
survey_frequency_per_box = df['box_name'].value_counts()

# Count of Retries (RR or INF or in) per Box
retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

# Duration per Line in Box
duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

# Total Duration per Box
total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

# Read and process both text files with the completed tasks info
completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

# Concatenate the dataframes
df_combined = pd.concat([df1, df2])

# Compute the metrics for the Summary sheet
completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

avg_time_between_boxes = box_changes['time_diff'].mean()
bad_weather_wait = box_changes[box_changes['time_diff'] > 1000]['time_diff'].mean()

# Assuming "BH_" and "CPT_" in box names indicate the type of the boxes
avg_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].mean()
avg_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].mean()

# Create a summary DataFrame
summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage,  total_distance_surveyed, avg_BH_box_time, avg_CPT_box_time, avg_time_between_boxes, bad_weather_wait]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Average BH Box Time (Minutes)', 'Average CPT Box Time (Minutes)', 'Average Time Between Boxes (Minutes)', 'Average Downtime due to Weather (Minutes)']
)

# Transpose the summary DataFrame
summary_df = summary_df.T



# ... Rest of the code is unchanged ...

# Write dataframes to different sheets in the Excel file
with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation')
    df_combined.to_excel(writer, sheet_name='Progress', index=False)

    # Create the 'History' sheet
    last_box_change_date = box_changes['log_off'].max()  # Get the most recent date from 'Time Between Boxes' sheet
    history_df = pd.DataFrame([last_box_change_date], columns=['Latest Date'])  # Create a DataFrame with the latest date
    history_df.to_excel(writer, sheet_name='History', index=False)  # Write the DataFrame to 'History' sheet


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\4281584382.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\4281584382.py:44: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\4281584382.py:110: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\4281584382.py:13

### Add storing Box Locations

In [14]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df

# Load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
    last_log_off = row['to']  

df['day'] = df['from'].dt.date

grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

total_distance_surveyed = df['distance'].sum()

total_lines_by_team = df.groupby('teams')['line'].count()

total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

total_distance_per_box = df.groupby('box_name')['distance'].sum()

survey_frequency_per_box = df['box_name'].value_counts()

retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

df_combined = pd.concat([df1, df2])

completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

avg_time_between_boxes = box_changes['time_diff'].mean()
bad_weather_wait = box_changes[box_changes['time_diff'] > 1000]['time_diff'].mean()

avg_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].mean()
avg_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].mean()

summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage,  total_distance_surveyed, avg_BH_box_time, avg_CPT_box_time, avg_time_between_boxes, bad_weather_wait]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Average BH Box Time (Minutes)', 'Average CPT Box Time (Minutes)', 'Average Time Between Boxes (Minutes)', 'Average Downtime due to Weather (Minutes)']
)

summary_df = summary_df.T

# Load box location data from csv
box_location_df = pd.read_csv('box_locations.csv', names=['Box Name', 'X', 'Y'])

with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation')
    df_combined.to_excel(writer, sheet_name='Progress', index=False)
    last_box_change_date = box_changes['log_off'].max()
    history_df = pd.DataFrame([last_box_change_date], columns=['Latest Date'])
    history_df.to_excel(writer, sheet_name='History', index=False)
    box_location_df.to_excel(writer, sheet_name='Box Location', index=False)


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3619594203.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3619594203.py:44: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3619594203.py:96: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\3619594203.py:112

In [15]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('marine_survey_analysis.xlsx')

# Get the sheet names
sheet_names = xls.sheet_names

# Print the sheet names
for sheet in sheet_names:
    print(sheet)


Summary
Teams
Total Distance per Box
Survey Frequency per Box
Retries per Box
Duration per Line in Box
Total Duration per Box
Time Between Boxes
Total Duration per Box with Relocation
Progress
History
Box Location


### adding column for plot in bi

In [1]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df

# Load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
    last_log_off = row['to']  

df['day'] = df['from'].dt.date

grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

total_distance_surveyed = df['distance'].sum()

total_lines_by_team = df.groupby('teams')['line'].count()

total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

total_distance_per_box = df.groupby('box_name')['distance'].sum()

survey_frequency_per_box = df['box_name'].value_counts()

retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

df_combined = pd.concat([df1, df2])

completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

avg_time_between_boxes = box_changes['time_diff'].mean()
bad_weather_wait = box_changes[box_changes['time_diff'] > 1000]['time_diff'].mean()

avg_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].mean()
avg_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].mean()

summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage,  total_distance_surveyed, avg_BH_box_time, avg_CPT_box_time, avg_time_between_boxes, bad_weather_wait]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Average BH Box Time (Minutes)', 'Average CPT Box Time (Minutes)', 'Average Time Between Boxes (Minutes)', 'Average Downtime due to Weather (Minutes)']
)

summary_df = summary_df.T

# Load box location data from csv
box_location_df = pd.read_csv('box_locations.csv', names=['Box Name', 'X', 'Y'])

# Define the completed boxes from the box_changes DataFrame
completed_boxes = box_changes['box_name'].unique()

# Add a new column to the box_location_df DataFrame, checking if the box has been completed
box_location_df['Completed'] = box_location_df['Box Name'].apply(lambda x: 'y' if x in completed_boxes else '')

with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance per Box')
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Total Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Relocation')
    df_combined.to_excel(writer, sheet_name='Progress', index=False)
    last_box_change_date = box_changes['log_off'].max()
    history_df = pd.DataFrame([last_box_change_date], columns=['Latest Date'])
    history_df.to_excel(writer, sheet_name='History', index=False)
    box_location_df.to_excel(writer, sheet_name='Box Location', index=False)


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_13392\2509176320.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_13392\2509176320.py:44: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_13392\2509176320.py:96: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_13392\2509176320.py

In [4]:
times_between_boxes = [56.47, 30.78, 23.40, 49.78, 8.77, 31.82, 39.33, 34.48, 33.08, 3201.05, 175.05, 36.48, 50.97, 24.50, 52.60, 33.68, 28.95, 38.78, 29.83, 32.10, 26.87, 28.43, 31.80, 44.95, 49.52, 31.65, 41.28, 112.02, 38.02, 41.90, 132.55, 31.90, 61.68, 42.45, 75.15, 30.05, 32.30, 45.88, 37.95, 28.30, 29, 31.07, 43.62, 36.80, 28.57, 22.83, 49.25, 33.78, 36.67, 34.75, 33.03, 30.33, 27.67, 35.80, 36.12, 43.50, 3420.13, 50.72, 48.68, 42.85, 55.87, 48.08, 37.83, 70.22, 82.43, 44.27, 58.07, 51.15, 94.88, 52.60, 57.48, 43.85, 37.63, 52.42, 39.72, 36.83]
sorted_times = sorted(times_between_boxes)

n = len(sorted_times)
if n % 2 == 0:  # if the length of the list is even
    median_1 = sorted_times[n//2]
    median_2 = sorted_times[n//2 - 1]
    median = (median_1 + median_2)/2
else:
    median = sorted_times[n//2]

print(median)

38.400000000000006


In [5]:
import numpy as np

# BH and CPT times as provided
bh_times = [225.12, 227.55, 200.72, 182.68, 266.03, 200, 169, 187.32, 192.28, 206.23, 210.95, 265.57, 211, 164.23, 161.07, 188.03, 198.85, 207.12, 182.55, 175.1, 202.22, 202.28]
cpt_times = [117.93, 854.4, 121.25, 161.08, 199.85, 95.77, 134.68, 125.85, 177.02, 123.1, 114.12, 123.88, 114.82, 127.45, 125.25, 155.95, 100.2, 120.55, 132.27, 104.78, 100.65, 450.82, 107.92, 147.35, 99.67, 168.68, 103.65, 129.15, 102.1, 135.27, 143.8, 242.7, 114.72, 94.38, 137.85, 103.38, 129.92, 186.82, 115.08, 124.07, 196.82, 150.95, 119.45, 155.37, 117.25, 157.77, 132.15, 123.15, 121.92, 91.57, 134.05, 122.97]

# Calculate the median
median_bh_time = np.median(bh_times)
median_cpt_time = np.median(cpt_times)

print(f"The median BH box time is {median_bh_time} minutes")
print(f"The median CPT box time is {median_cpt_time} minutes")


The median BH box time is 200.36 minutes
The median CPT box time is 124.66 minutes


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_16272\2618215969.py:43: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_16272\2618215969.py:44: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_16272\2618215969.py:113: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_16272\2618215969.p

# Latest with travel distances.

In [16]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    completed_tasks = df['task'].unique().tolist()
    return completed_tasks

def parse_line(line, completed_tasks):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        return box, line, area, 'y' if f'{box}_{line}' in completed_tasks else ''
    except ValueError:
        return None, None, None, None

def process_file(filename, completed_tasks):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks) for line in lines if parse_line(line, completed_tasks) != (None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed'])
    return df



############################


############################

# Load the data
df = pd.read_excel('output.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

##################

##################

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
    last_log_off = row['to']  

df['day'] = df['from'].dt.date

grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

total_distance_surveyed = df['distance'].sum()

total_lines_by_team = df.groupby('teams')['line'].count()

total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

total_distance_per_box = df.groupby('box_name')['distance'].sum()

survey_frequency_per_box = df['box_name'].value_counts()

retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

completed_tasks = read_output_file('output.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

df_combined = pd.concat([df1, df2])

completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

median_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].median()
median_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].median()
median_time_between_boxes = box_changes['time_diff'].median()




# Load box location data from csv
box_location_df = pd.read_csv('box_locations.csv', names=['Box Name', 'X', 'Y'])

# Define the completed boxes from the box_changes DataFrame
completed_boxes = box_changes['box_name'].unique()

# Add a new column to the box_location_df DataFrame, checking if the box has been completed
box_location_df['Completed'] = box_location_df['Box Name'].apply(lambda x: 'y' if x in completed_boxes else '')

###########################################


# Function to calculate distance between two points in a Cartesian coordinate system
def calculate_distance(x1, y1, x2, y2):
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 1000  # Convert to km
    return distance

# Merge box locations to the 'Time Between Boxes' dataframe
df_box_changes_with_location = box_changes.merge(box_location_df, 
                                        left_on='box_name', 
                                        right_on='Box Name', 
                                        how='left')

df_box_changes_with_location = df_box_changes_with_location.merge(box_location_df, 
                            left_on='next_box', 
                            right_on='Box Name', 
                            how='left', 
                            suffixes=('_current', '_next'))

# Calculate distance
df_box_changes_with_location['distance'] = df_box_changes_with_location.apply(lambda row: calculate_distance(row['X_current'], row['Y_current'], row['X_next'], row['Y_next']), axis=1)

# Create 'Travel Box to Box' dataframe
df_travel_box_to_box = df_box_changes_with_location[['box_name', 'next_box', 'distance']]

totaldistance = df_travel_box_to_box['distance'].sum()

summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage, total_distance_surveyed,  totaldistance, median_BH_box_time, median_CPT_box_time, median_time_between_boxes]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Total Distance Box-to-Box', 'Median BH Box Time (Minutes)', 'Median CPT Box Time (Minutes)', 'Median Time Between Boxes (Minutes)']
)


summary_df = summary_df.T


##################################################

with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance logged per Box')
    df_travel_box_to_box.to_excel(writer, sheet_name='Travel Box to Box', index=False)
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Logged Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Line Relocation')
    box_location_df.to_excel(writer, sheet_name='Box Locations', index=False)
    df_combined.to_excel(writer, sheet_name='Progress', index=False)
    last_box_change_date = box_changes['log_off'].max()
    history_df = pd.DataFrame([last_box_change_date], columns=['Latest Date'])
    history_df.to_excel(writer, sheet_name='History', index=False)
    


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1206103077.py:50: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['from'] = pd.to_datetime(df['from'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1206103077.py:51: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['to'] = pd.to_datetime(df['to'])
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1206103077.py:107: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_6392\1206103077.py:12

In [14]:
import pandas as pd
import numpy as np

def duration_to_minutes(duration):
    h, m, s = map(int, duration.split(':'))
    return h * 60 + m + s / 60

def read_output_file(filename):
    df = pd.read_excel(filename)
    df['task'] = df['box_name'] + '_' + df['line']
    return df[['task', 'to']]

def parse_line(line, completed_tasks_df):
    try:
        box_line, area = line.strip().split(',')
        box, line = box_line.rsplit('_', 1)
        task = f'{box}_{line}'
        completion_date = completed_tasks_df.loc[completed_tasks_df['task'] == task, 'to']
        return box, line, area, 'y' if task in completed_tasks_df['task'].values else '', completion_date.iloc[0] if len(completion_date) > 0 else None
    except ValueError:
        return None, None, None, None, None

def process_file(filename, completed_tasks_df):
    with open(filename, 'r') as file:
        lines = file.readlines()
    parsed_lines = [parse_line(line, completed_tasks_df) for line in lines if parse_line(line, completed_tasks_df) != (None, None, None, None, None)]
    df = pd.DataFrame(parsed_lines, columns=['box', 'line', 'area', 'completed', 'completion_date'])
    return df

# Modify the below section of the code
completed_tasks_df = read_output_file('output2.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks_df)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks_df)

df_combined = pd.concat([df1, df2])

# Convert 'completion_date' to datetime and then to string
df_combined['completion_date'] = pd.to_datetime(df_combined['completion_date']).dt.strftime('%Y-%m-%d %H:%M:%S')


############################


############################

# Load the data
df = pd.read_excel('output2.xlsx')

# Ensure 'distance' is a float
df['distance'] = df['distance'].str.replace(' km','').astype(float)

# Rename 'operator' to 'teams'
df.rename(columns={'operator': 'teams'}, inplace=True)

# Sort strings alphabetically within each teams
df['teams'] = df['teams'].apply(lambda x: '/'.join(sorted(x.split('/'))))

# Convert 'duration' and 'from', 'to' to minutes
df['duration_minutes'] = df['duration'].apply(duration_to_minutes)
df['from'] = pd.to_datetime(df['from'], format="%d/%m/%Y %H:%M:%S")
df['to'] = pd.to_datetime(df['to'], format="%d/%m/%Y %H:%M:%S")


# Sort DataFrame by 'from' in ascending order
df.sort_values('from', inplace=True)

# Initialize an empty DataFrame to hold box change data
box_changes = pd.DataFrame(columns=['box_name', 'log_off', 'next_box', 'log_on', 'time_diff'])

##################

##################

last_box = None
last_log_off = None

for _, row in df.iterrows():
    if row['box_name'] != last_box:
        if last_box is not None:
            next_log_on = df[(df['box_name'] == row['box_name']) & (df['from'] > last_log_off)]['from'].min()
            if pd.isnull(next_log_on):  
                continue
            time_diff = (next_log_on - last_log_off).total_seconds() / 60
            box_changes = pd.concat([box_changes, pd.DataFrame([{
                'box_name': last_box,
                'log_off': last_log_off.strftime('%Y-%m-%d %H:%M:%S'),
                'next_box': row['box_name'],
                'log_on': next_log_on.strftime('%Y-%m-%d %H:%M:%S'),
                'time_diff': time_diff
            }])])
        last_box = row['box_name']
    last_log_off = row['to']  

df['day'] = df['from'].dt.date

grouped = df.groupby(['box_name', 'day']).agg({'from': 'min', 'to': 'max'})

grouped['duration'] = (grouped['to'] - grouped['from']).dt.total_seconds() / 60

total_duration_per_box_with_relocation = grouped.groupby('box_name')['duration'].sum()

# fastest box
fastest_bh_box = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].idxmin()
fastest_cpt_box = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].idxmin()
fastest_bh_box_time = total_duration_per_box_with_relocation.loc[fastest_bh_box]
fastest_cpt_box_time = total_duration_per_box_with_relocation.loc[fastest_cpt_box]


df['from'] = df['from'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['to'] = df['to'].dt.strftime('%Y-%m-%d %H:%M:%S')

total_distance_surveyed = df['distance'].sum()

total_lines_by_team = df.groupby('teams')['line'].count()

total_boxes_by_team = df.groupby('teams')['box_name'].nunique()

team_efficiency = pd.concat([total_lines_by_team, total_boxes_by_team], axis=1)
team_efficiency.columns = ['Total Lines', 'Total Boxes']

total_distance_per_box = df.groupby('box_name')['distance'].sum()

survey_frequency_per_box = df['box_name'].value_counts()

retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)

duration_per_line_in_box = df.groupby(['box_name', 'line'])['duration_minutes'].mean()

total_duration_per_box = df.groupby('box_name')['duration_minutes'].sum()

completed_tasks = read_output_file('output2.xlsx')
df1 = process_file('Boreholes_RunLines_20230617.txt', completed_tasks)
df2 = process_file('CPT_RunLines_20230617.txt', completed_tasks)

df_combined = pd.concat([df1, df2])

completed_lines = df_combined[df_combined['completed'] == 'y'].shape[0]
total_lines = df_combined.shape[0]
completion_percentage = completed_lines / total_lines * 100

retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].shape[0]
retries_percentage = retries / completed_lines * 100

median_BH_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('BH_')].median()
median_CPT_box_time = total_duration_per_box_with_relocation[total_duration_per_box_with_relocation.index.str.startswith('CPT_')].median()
median_time_between_boxes = box_changes['time_diff'].median()

############


# Load box location data from csv
box_location_df = pd.read_csv('box_locations.csv', names=['Box Name', 'X', 'Y'])

# Define the completed boxes from the box_changes DataFrame
completed_boxes = box_changes['box_name'].unique()

# Add a new column to the box_location_df DataFrame, checking if the box has been completed
box_location_df['Completed'] = box_location_df['Box Name'].apply(lambda x: 'y' if x in completed_boxes else '')

###########################################


# Convert 'from' and 'to' to datetime
df['from'] = pd.to_datetime(df['from'])
df['to'] = pd.to_datetime(df['to'])

# Split the 'teams' column into individual operators and explode
df['operators'] = df['teams'].str.split('/')
df_exploded = df.explode('operators')

# Calculate the first log on and last log off for each operator
log_on_off_times = df_exploded.groupby('operators').agg({'from': 'min', 'to': 'max'})

# Rename the columns for clarity
log_on_off_times.columns = ['First Log On', 'Last Log Off']

# Convert datetime to string
log_on_off_times['First Log On'] = log_on_off_times['First Log On'].dt.strftime('%Y-%m-%d %H:%M:%S')
log_on_off_times['Last Log Off'] = log_on_off_times['Last Log Off'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Merge team_efficiency with log_on_off_times
team_efficiency_exploded = df_exploded.groupby('operators').agg({'line': 'count', 'box_name': 'nunique'})
team_efficiency_exploded.columns = ['Total Lines', 'Total Boxes']

team_efficiency_exploded = team_efficiency_exploded.merge(log_on_off_times, left_index=True, right_index=True)


# ...
# ...



########################################


# Function to calculate distance between two points in a Cartesian coordinate system
def calculate_distance(x1, y1, x2, y2):
    distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / 1000  # Convert to km
    return distance

# Merge box locations to the 'Time Between Boxes' dataframe
df_box_changes_with_location = box_changes.merge(box_location_df, 
                                        left_on='box_name', 
                                        right_on='Box Name', 
                                        how='left')

df_box_changes_with_location = df_box_changes_with_location.merge(box_location_df, 
                            left_on='next_box', 
                            right_on='Box Name', 
                            how='left', 
                            suffixes=('_current', '_next'))

# Calculate distance
df_box_changes_with_location['distance'] = df_box_changes_with_location.apply(lambda row: calculate_distance(row['X_current'], row['Y_current'], row['X_next'], row['Y_next']), axis=1)

# Create 'Travel Box to Box' dataframe
df_travel_box_to_box = df_box_changes_with_location[['box_name', 'next_box', 'distance']]

totaldistance = df_travel_box_to_box['distance'].sum()

summary_df = pd.DataFrame(
    data=np.array([completion_percentage, completed_lines, retries_percentage, total_distance_surveyed,  totaldistance, median_BH_box_time, median_CPT_box_time, median_time_between_boxes, fastest_bh_box_time, fastest_cpt_box_time]).reshape(1, -1),
    columns=['Completion Percentage', 'Completed Lines', 'Infill Percentage', 'Total Distance Surveyed (Km)', 'Total Distance Box-to-Box', 'Median BH Box Time (Minutes)', 'Median CPT Box Time (Minutes)', 'Median Time Between Boxes (Minutes)', 'Fastest BH Box Time (Minutes)', 'Fastest CPT Box Time (Minutes)']
)


completed_boxes = df_combined.groupby('box')['completed'].apply(lambda x: (x == 'y').all())
completed_boxes_count = completed_boxes.sum()

summary_df.insert(2, 'Completed Boxes', completed_boxes_count)



summary_df = summary_df.T


##################################################

with pd.ExcelWriter('marine_survey_analysis.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', header=False)
    team_efficiency.to_excel(writer, sheet_name='Teams')
    team_efficiency_exploded.to_excel(writer, sheet_name='Operator Schedule')
    total_distance_per_box.to_excel(writer, sheet_name='Total Distance logged per Box')
    df_travel_box_to_box.to_excel(writer, sheet_name='Travel Box to Box', index=False)
    survey_frequency_per_box.to_excel(writer, sheet_name='Survey Frequency per Box')
    retries_per_box.to_excel(writer, sheet_name='Retries per Box')
    duration_per_line_in_box.to_excel(writer, sheet_name='Duration per Line in Box')
    total_duration_per_box.to_excel(writer, sheet_name='Logged Duration per Box')
    box_changes.to_excel(writer, sheet_name='Time Between Boxes', index=False)
    total_duration_per_box_with_relocation.to_excel(writer, sheet_name='Total Duration per Box with Line Relocation')
    box_location_df.to_excel(writer, sheet_name='Box Locations', index=False)
    df_combined.to_excel(writer, sheet_name='Progress', index=False)
    last_box_change_date = box_changes['log_off'].max()
    history_df = pd.DataFrame([last_box_change_date], columns=['Latest Date'])
    history_df.to_excel(writer, sheet_name='History', index=False)
    


C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_3320\3666248126.py:38: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_combined['completion_date'] = pd.to_datetime(df_combined['completion_date']).dt.strftime('%Y-%m-%d %H:%M:%S')
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_3320\3666248126.py:125: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries_per_box = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)]['box_name'].value_counts().reindex(df['box_name'].unique(), fill_value=0)
C:\Users\Edin.Smlatic\AppData\Local\Temp\ipykernel_3320\3666248126.py:141: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retries = df[df['line'].str.contains('(_RR|_INF|_in)', case=False)].sha

# Predict

In [11]:
# Required Libraries
import pandas as pd
import numpy as np

# Load Box Locations data from Excel file
box_locations = pd.read_excel('marine_survey_analysis.xlsx', sheet_name='Box Locations')

# Convert 'X' and 'Y' coordinates to kilometers
box_locations['X'] /= 1000
box_locations['Y'] /= 1000


# Function to calculate Cartesian distance
def cartesian_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Predict Journey Time Function
def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, speed_outside_range, min_distance, max_distance, box_locations):
    
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0
    total_time = 0
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            total_time += median_survey_time_BH
        elif 'CPT' in box_type:
            total_time += median_survey_time_CPT
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if min_distance <= distance <= max_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # Use a constant speed for distances outside the observed range
                travel_time = distance / speed_outside_range * 60  # convert from hours to minutes
            
            # Add the travel time to the total time
            total_time += travel_time
    
    # Return the total time
    return total_time

################ Parameters

# Median survey times for BH and CPT boxes in minutes. These values are based on historical data.
median_survey_time_BH = 190.33
median_survey_time_CPT = 121.92

# Parameters for the fixed + variable time model for travel times within the observed range in minutes.
# The fixed time is a constant time required for every travel, and the variable time is an additional time per kilometer.
fixed_time = 8.77
# If speed is in km/hour
speed_inside_range = 9.46   # fill in with the known speed
variable_time = 60 / speed_inside_range  # convert from km/hour to minutes/km



# Speed in km/h to be used for distances outside the observed range.
speed_outside_range = 3.38

# Observed range of distances between boxes in kilometers. Travel times for distances within this range are calculated 
# using the fixed + variable time model, and for distances outside this range are calculated using a constant speed.
min_distance = 0.5
max_distance = 11.23


###########



# List of Boxes
boxes = ['CPT_055', 'CPT_202']


"""
# Load Boxes from Excel file
boxes_df = pd.read_excel('/mnt/data/boxes.xlsx')

# Check if there are coordinates provided in the Excel file
if 'X' in boxes_df.columns and 'Y' in boxes_df.columns:
    # If yes, convert the DataFrame to a list of tuples
    boxes_df['X'] /= 1000  # Convert X coordinates to kilometers
    boxes_df['Y'] /= 1000  # Convert Y coordinates to kilometers
    boxes = list(boxes_df.itertuples(index=False, name=None))
else:
    # If not, convert the DataFrame to a list of box names
    boxes = boxes_df.iloc[:, 0].tolist()

"""




# Predict Journey Time
journey_time = predict_journey_time(
    boxes=boxes, 
    median_survey_time_BH=median_survey_time_BH, 
    median_survey_time_CPT=median_survey_time_CPT, 
    fixed_time=fixed_time, 
    variable_time=variable_time, 
    speed_outside_range=speed_outside_range, 
    min_distance=min_distance, 
    max_distance=max_distance, 
    box_locations=box_locations
)

print(f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.")



Estimated Total Journey Time for the given list of boxes: 272.66672907544256 minutes or approximately 4.54 hours.


In [12]:
# Predict Journey Time Function
def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, speed_outside_range, min_distance, max_distance, box_locations):
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0 and create an empty list to store travel times
    total_time = 0
    travel_times = []
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            total_time += median_survey_time_BH
        elif 'CPT' in box_type:
            total_time += median_survey_time_CPT
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if min_distance <= distance <= max_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # Use a constant speed for distances outside the observed range
                travel_time = distance / speed_outside_range * 60  # convert from hours to minutes
            
            # Add the travel time to the total time and to the list of travel times
            total_time += travel_time
            travel_times.append(travel_time)
    
    # Return the total time and the list of travel times
    return total_time, travel_times

# Predict Journey Time and Travel Times
journey_time, travel_times = predict_journey_time(
    boxes=boxes, 
    median_survey_time_BH=median_survey_time_BH, 
    median_survey_time_CPT=median_survey_time_CPT, 
    fixed_time=fixed_time, 
    variable_time=variable_time, 
    speed_outside_range=speed_outside_range, 
    min_distance=min_distance, 
    max_distance=max_distance, 
    box_locations=box_locations
)

print(f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.")
print("Estimated Travel Times between consecutive boxes:", travel_times)


Estimated Total Journey Time for the given list of boxes: 272.66672907544256 minutes or approximately 4.54 hours.
Estimated Travel Times between consecutive boxes: [28.826729075442536]


In [ ]:
""""ArithmeticError

Speed notes, 3km top speed was 6 knot. took almost 1km, 4 knot took 500 meters. hard to know for sure...


"""

In [6]:
# Required Libraries
import pandas as pd
import numpy as np

# Load Box Locations data from Excel file
box_locations = pd.read_excel('marine_survey_analysis.xlsx', sheet_name='Box Locations')

# Convert 'X' and 'Y' coordinates to kilometers
box_locations['X'] /= 1000
box_locations['Y'] /= 1000

# Function to calculate Cartesian distance
def cartesian_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Predict Journey Time Function
def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, speed_outside_range, min_distance, max_distance, box_locations):
    
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0
    total_time = 0
    travel_times = []
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            total_time += median_survey_time_BH
        elif 'CPT' in box_type:
            total_time += median_survey_time_CPT
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if min_distance <= distance <= max_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # Use a constant speed for distances outside the observed range
                travel_time = distance / speed_outside_range * 60  # convert from hours to minutes
            
            # Add the travel time to the total time
            total_time += travel_time
            travel_times.append(travel_time)
    
    # Return the total time
    return total_time, travel_times


################ Parameters

# Median survey times for BH and CPT boxes in minutes. These values are based on historical data.
median_survey_time_BH = 170
median_survey_time_CPT = 100

# Parameters for the fixed + variable time model for travel times within the observed range in minutes.
# The fixed time is a constant time required for every travel, and the variable time is an additional time per kilometer.
fixed_time = 8.77
# If speed is in km/hour
speed_inside_range = 9.46   # fill in with the known speed
variable_time = 60 / speed_inside_range  # convert from km/hour to minutes/km

# Speed in km/h to be used for distances outside the observed range.
speed_outside_range = 10

# Observed range of distances between boxes in kilometers. Travel times for distances within this range are calculated 
# using the fixed + variable time model, and for distances outside this range are calculated using a constant speed.
min_distance = 0.5
max_distance = 11.23

# List of Boxes
boxes = ['CPT_202', 'BH_064', 'CPT_101']

# Predict Journey Time
journey_time, travel_times = predict_journey_time(
    boxes=boxes, 
    median_survey_time_BH=median_survey_time_BH, 
    median_survey_time_CPT=median_survey_time_CPT, 
    fixed_time=fixed_time, 
    variable_time=variable_time, 
    speed_outside_range=speed_outside_range, 
    min_distance=min_distance, 
    max_distance=max_distance, 
    box_locations=box_locations
)

print(f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.")
print("Estimated Travel Times between consecutive boxes:", travel_times)


Estimated Total Journey Time for the given list of boxes: 410.76833333197743 minutes or approximately 6.85 hours.
Estimated Travel Times between consecutive boxes: [18.922945699051052, 21.845387632926375]


In [3]:
# Required Libraries
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, speed_outside_range, min_distance, max_distance, box_locations, start_time):

    # Convert start_time from string to datetime
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    
    # (the rest of your function goes here)

    # Return the total time and end time
    return total_time, travel_times, start_time + timedelta(minutes=total_time)


# Load Box Locations data from Excel file
box_locations = pd.read_excel('marine_survey_analysis.xlsx', sheet_name='Box Locations')

# Convert 'X' and 'Y' coordinates to kilometers
box_locations['X'] /= 1000
box_locations['Y'] /= 1000

# Function to calculate Cartesian distance
def cartesian_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, speed_outside_range, min_distance, max_distance, box_locations, start_time):
    
    # Convert start_time from string to datetime
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0
    total_time = 0
    travel_times = []
    
    # Initialize journey summary
    journey_summary = []
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            box_time = median_survey_time_BH
            total_time += box_time
        elif 'CPT' in box_type:
            box_time = median_survey_time_CPT
            total_time += box_time
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        box_start_time = start_time + timedelta(minutes=total_time - box_time)
        box_end_time = start_time + timedelta(minutes=total_time)
        
        # Add the box details to the journey summary
        journey_summary.append({
            'box_name': box_name,
            'start_time': box_start_time,
            'end_time': box_end_time,
            'box_time': box_time,
        })
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if min_distance <= distance <= max_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # Use a constant speed for distances outside the observed range
                travel_time = distance / speed_outside_range * 60  # convert from hours to minutes
            
            # Add the travel time to the total time
            total_time += travel_time
            travel_times.append(travel_time)
            
            # Add the travel time to the journey summary
            journey_summary[-1]['travel_time_to_next_box'] = travel_time
    
    # Return the total time, end time and journey summary
    return total_time, travel_times, start_time + timedelta(minutes=total_time), journey_summary



################ Parameters

# Median survey times for BH and CPT boxes in minutes. These values are based on historical data.
median_survey_time_BH = 170
median_survey_time_CPT = 100

# Parameters for the fixed + variable time model for travel times within the observed range in minutes.
# The fixed time is a constant time required for every travel, and the variable time is an additional time per kilometer.
fixed_time = 8.77
# If speed is in km/hour
speed_inside_range = 9.46   # fill in with the known speed
variable_time = 60 / speed_inside_range  # convert from km/hour to minutes/km

# Speed in km/h to be used for distances outside the observed range.
speed_outside_range = 10

# Observed range of distances between boxes in kilometers. Travel times for distances within this range are calculated 
# using the fixed + variable time model, and for distances outside this range are calculated using a constant speed.
min_distance = 0.5
max_distance = 11.23

# List of Boxes
boxes = ['BH_064', 'CPT_101', 'CPT_085', 'CPT_246', 'BH_125', 'CPT_068', 'CPT_159']

# Start time
start_time = "06:37:54"

# Predict Journey Time
journey_time, travel_times, end_time, journey_summary = predict_journey_time(
    boxes=boxes, 
    median_survey_time_BH=median_survey_time_BH, 
    median_survey_time_CPT=median_survey_time_CPT, 
    fixed_time=fixed_time, 
    variable_time=variable_time, 
    speed_outside_range=speed_outside_range, 
    min_distance=min_distance, 
    max_distance=max_distance, 
    box_locations=box_locations, 
    start_time=start_time
)

print(f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.")
print("Estimated Travel Times between consecutive boxes:", travel_times)
print(f"Estimated End Time for the journey: {end_time.strftime('%H:%M:%S')}")

print("\nJourney Summary:")
for i, details in enumerate(journey_summary):
    print(f"\nBox {i+1}: {details['box_name']}")
    print(f"Start Time: {details['start_time'].strftime('%H:%M:%S')}")
    print(f"End Time: {details['end_time'].strftime('%H:%M:%S')}")
    print(f"Time Spent on Box: {details['box_time']} minutes")
    if 'travel_time_to_next_box' in details:
        print(f"Travel Time to Next Box: {details['travel_time_to_next_box']} minutes")


print(f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.")
print("Estimated Travel Times between consecutive boxes:", travel_times)
print(f"Estimated End Time for the journey: {end_time.strftime('%H:%M:%S')}")

print("\nJourney Summary:")
for i, details in enumerate(journey_summary):
    print(f"\nBox {i+1}: {details['box_name']}")
    print(f"Start Time: {details['start_time'].strftime('%H:%M:%S')}")
    print(f"End Time: {details['end_time'].strftime('%H:%M:%S')}")
    print(f"Time Spent on Box: {details['box_time']} minutes")
    if 'travel_time_to_next_box' in details:
        print(f"Travel Time to Next Box: {details['travel_time_to_next_box']} minutes")


# Convert summary to DataFrame
summary_df = pd.DataFrame(journey_summary)

# Convert datetime columns to strings for better readability in Excel
summary_df['start_time'] = summary_df['start_time'].apply(lambda x: x.strftime('%H:%M:%S'))
summary_df['end_time'] = summary_df['end_time'].apply(lambda x: x.strftime('%H:%M:%S'))

# Write to Excel
summary_df.to_excel('journey_summary.xlsx', index=False)

Estimated Total Journey Time for the given list of boxes: 977.0588068558676 minutes or approximately 16.28 hours.
Estimated Travel Times between consecutive boxes: [21.845387632926375, 21.55370646560571, 23.040613204876607, 22.952249535051784, 23.040613204875037, 24.626236812532035]
Estimated End Time for the journey: 22:54:57

Journey Summary:

Box 1: BH_064
Start Time: 06:37:54
End Time: 09:27:54
Time Spent on Box: 170 minutes
Travel Time to Next Box: 21.845387632926375 minutes

Box 2: CPT_101
Start Time: 09:49:44
End Time: 11:29:44
Time Spent on Box: 100 minutes
Travel Time to Next Box: 21.55370646560571 minutes

Box 3: CPT_085
Start Time: 11:51:17
End Time: 13:31:17
Time Spent on Box: 100 minutes
Travel Time to Next Box: 23.040613204876607 minutes

Box 4: CPT_246
Start Time: 13:54:20
End Time: 15:34:20
Time Spent on Box: 100 minutes
Travel Time to Next Box: 22.952249535051784 minutes

Box 5: BH_125
Start Time: 15:57:17
End Time: 18:47:17
Time Spent on Box: 170 minutes
Travel Time t

# GUI test

In [8]:
# Required Libraries
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime, timedelta

# Function to calculate Cartesian distance
def cartesian_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Journey Time Prediction Function
def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, min_distance, box_locations, start_time):

    # Convert start_time from string to datetime
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0
    total_time = 0
    travel_times = []
    
    # Initialize journey summary
    journey_summary = []
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            box_time = median_survey_time_BH
            total_time += box_time
        elif 'CPT' in box_type:
            box_time = median_survey_time_CPT
            total_time += box_time
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        box_start_time = start_time + timedelta(minutes=total_time - box_time)
        box_end_time = start_time + timedelta(minutes=total_time)
        
        # Add the box details to the journey summary
        journey_summary.append({
            'box_name': box_name,
            'start_time': box_start_time,
            'end_time': box_end_time,
            'box_time': box_time,
        })
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if distance >= min_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # If distance is less than min_distance, only the box_time is considered, travel_time is assumed to be negligible
                travel_time = 0
            
            # Add the travel time to the total time
            total_time += travel_time
            travel_times.append(travel_time)
            
            # Add the travel time to the journey summary
            journey_summary[-1]['travel_time_to_next_box'] = travel_time
    
    # Return the total time, end time and journey summary
    return total_time, travel_times, start_time + timedelta(minutes=total_time), journey_summary

# Database Loading Function
def load_database():
    global box_locations
    filepath = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
    box_locations = pd.read_excel(filepath, sheet_name='Box Locations')
    box_locations['X'] /= 1000
    box_locations['Y'] /= 1000
    messagebox.showinfo('Database Loaded', 'Survey Log has been loaded successfully!')

# Journey Start Function
def add_journey_stop():
    journey_stop_window = tk.Toplevel(root)
    journey_stop_window.geometry('+%d+%d' % (root.winfo_rootx()+50, root.winfo_rooty()+50))  # opens the new window slightly offset from the main window
    journey_stop_window.title('Add Journey Stops')
    
    # Add journey stop fields
    journey_stops.append(tk.StringVar(journey_stop_window, value=''))
    for i, stop in enumerate(journey_stops):
        tk.Label(journey_stop_window, text=f'Stop {i+1}').grid(row=i, column=0)
        tk.Entry(journey_stop_window, textvariable=stop).grid(row=i, column=1)
    
    # Add "Add Another Stop" button
    tk.Button(journey_stop_window, text="Add Another Stop", command=lambda: add_another_stop(journey_stop_window)).grid(row=len(journey_stops), column=0)

# Function to add another stop
def add_another_stop(window):
    journey_stops.append(tk.StringVar(window, value=''))
    tk.Label(window, text=f'Stop {len(journey_stops)}').grid(row=len(journey_stops)-1, column=0)
    tk.Entry(window, textvariable=journey_stops[-1]).grid(row=len(journey_stops)-1, column=1)
    window.grid()


# Parameters Function
def add_parameters():
    parameters_window = tk.Toplevel(root)
    parameters_window.title('Parameters')
    
    # Add parameter fields
    tk.Label(parameters_window, text='Median survey time for BH boxes (minutes)').grid(row=0, column=0)
    tk.Entry(parameters_window, textvariable=median_survey_time_BH).grid(row=0, column=1)
    tk.Label(parameters_window, text='Median survey time for CPT boxes (minutes)').grid(row=1, column=0)
    tk.Entry(parameters_window, textvariable=median_survey_time_CPT).grid(row=1, column=1)
    tk.Label(parameters_window, text='Fixed time (minutes)').grid(row=2, column=0)
    tk.Entry(parameters_window, textvariable=fixed_time).grid(row=2, column=1)
    tk.Label(parameters_window, text='Speed in knots').grid(row=3, column=0)
    tk.Entry(parameters_window, textvariable=speed_in_knots).grid(row=3, column=1)
    tk.Label(parameters_window, text='Minimum distance (kilometers)').grid(row=4, column=0)
    tk.Entry(parameters_window, textvariable=min_distance).grid(row=4, column=1)

# Run Prediction Function
def run_prediction():
    boxes = [stop.get() for stop in journey_stops]
    variable_time = 60 / (float(speed_in_knots.get()) * 1.852)  # convert from knots to km/hr and then to min/km
    journey_time, travel_times, end_time, journey_summary = predict_journey_time(
        boxes, float(median_survey_time_BH.get()), float(median_survey_time_CPT.get()), 
        float(fixed_time.get()), variable_time, float(min_distance.get()), box_locations, start_time.get()
    )

    # Print results
    result_text = f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.\n"
    result_text += "Estimated Travel Times between consecutive boxes: " + ', '.join(str(t) for t in travel_times) + "\n"
    result_text += f"Estimated End Time for the journey: {end_time.strftime('%H:%M:%S')}\n"
    result_text += "\nJourney Summary:"
    for i, details in enumerate(journey_summary):
        result_text += f"\nBox {i+1}: {details['box_name']}\n"
        result_text += f"Start Time: {details['start_time'].strftime('%H:%M:%S')}\n"
        result_text += f"End Time: {details['end_time'].strftime('%H:%M:%S')}\n"
        result_text += f"Time Spent on Box: {details['box_time']} minutes\n"
        if 'travel_time_to_next_box' in details:
            result_text += f"Travel Time to Next Box: {details['travel_time_to_next_box']} minutes\n"
    result_label['text'] = result_text
    save_button.pack()

def save_excel():
    # Convert summary to DataFrame
    summary_df = pd.DataFrame(journey_summary)
    # Convert datetime columns to strings for better readability in Excel
    summary_df['start_time'] = summary_df['start_time'].apply(lambda x: x.strftime('%H:%M:%S'))
    summary_df['end_time'] = summary_df['end_time'].apply(lambda x: x.strftime('%H:%M:%S'))
    # Ask for save location and write to Excel
    filename = filedialog.asksaveasfilename(defaultextension=".xlsx")
    summary_df.to_excel(filename, index=False)

root = tk.Tk()

file_button = tk.Button(root, text="Select Survey Log", command=load_database)
file_button.pack()

journey_stops = []
add_box_button = tk.Button(root, text="Journey Start", command=add_journey_stop)
add_box_button.pack()

start_time_label = tk.Label(root, text="Start Time (HH:MM:SS)")
start_time_label.pack()
start_time = tk.StringVar(root, value='HH:MM:SS')
start_time_entry = tk.Entry(root, textvariable=start_time)
start_time_entry.pack()

params_button = tk.Button(root, text="Parameters", command=add_parameters)
params_button.pack()

run_button = tk.Button(root, text="Estimate ETA", command=run_prediction)
run_button.pack()

result_label = tk.Label(root, text="")
result_label.pack()

save_button = tk.Button(root, text="Export to Excel", command=save_excel)
# save_button will be packed after running the prediction

# Initialize parameters
median_survey_time_BH = tk.StringVar(root, value='170')
median_survey_time_CPT = tk.StringVar(root, value='100')
fixed_time = tk.StringVar(root, value='8.77')
speed_in_knots = tk.StringVar(root, value='9.46')
min_distance = tk.StringVar(root, value='0.5')

root.mainloop()


In [13]:
# Required Libraries
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime, timedelta

# Function to calculate Cartesian distance
def cartesian_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Journey Time Prediction Function
def predict_journey_time(boxes, median_survey_time_BH, median_survey_time_CPT, 
                         fixed_time, variable_time, min_distance, box_locations, start_time):

    # Convert start_time from string to datetime
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    
    # Convert box_locations DataFrame to dictionary
    box_dict = box_locations.set_index('Box Name')[['X', 'Y']].T.to_dict('list')
    
    # Initialize total time to 0
    total_time = 0
    travel_times = []
    
    # Initialize journey summary
    global journey_summary
    journey_summary = []
    
    # Iterate over the list of boxes
    for i in range(len(boxes)):
        
        # Get the box name and coordinates (if provided)
        if isinstance(boxes[i], str):
            # Look up the coordinates in box_dict
            box_name = boxes[i]
            if box_name not in box_dict:
                raise ValueError(f"Box name {box_name} not found in 'my_list'. Please provide coordinates.")
            box_coords = box_dict[box_name]
        else:
            # Use provided coordinates
            box_name, box_coords = boxes[i]
        
        # Get the type of the current box
        box_type = box_name[:3]
        
        # Add the time to survey the current box to the total time
        if 'BH' in box_type:
            box_time = median_survey_time_BH
            total_time += box_time
        elif 'CPT' in box_type:
            box_time = median_survey_time_CPT
            total_time += box_time
        else:
            raise ValueError(f"Unknown box type: {box_type}. Box names should start with 'BH' or 'CPT'.")
        
        box_start_time = start_time + timedelta(minutes=total_time - box_time)
        box_end_time = start_time + timedelta(minutes=total_time)
        
        # Add the box details to the journey summary
        journey_summary.append({
            'box_name': box_name,
            'start_time': box_start_time,
            'end_time': box_end_time,
            'box_time': box_time,
        })
        
        # Add the time to travel to the next box to the total time (if there is a next box)
        if i < len(boxes) - 1:
            # Get the coordinates of the next box
            next_box_coords = boxes[i + 1][1] if isinstance(boxes[i + 1], tuple) else box_dict[boxes[i + 1]]
            
            # Calculate the distance to the next box
            distance = cartesian_distance(*box_coords, *next_box_coords)
            
            # Calculate the travel time based on the distance
            if distance >= min_distance:
                # Use the fixed + variable time model for distances within the observed range
                travel_time = fixed_time + distance * variable_time
            else:
                # If distance is less than min_distance, only the box_time is considered, travel_time is assumed to be negligible
                travel_time = 0
            
            # Add the travel time to the total time
            total_time += travel_time
            travel_times.append(travel_time)
            
            # Add the travel time to the journey summary
            journey_summary[-1]['travel_time_to_next_box'] = travel_time
    
    # Return the total time, end time and journey summary
    return total_time, travel_times, start_time + timedelta(minutes=total_time), journey_summary

# Database Loading Function
def load_database():
    global box_locations
    filepath = filedialog.askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
    box_locations = pd.read_excel(filepath, sheet_name='Box Locations')
    box_locations['X'] /= 1000
    box_locations['Y'] /= 1000
    messagebox.showinfo('Database Loaded', 'Survey Log has been loaded successfully!')

## Journey Start Function
def add_journey_stop():
    journey_stop_frame = tk.Frame(root)
    journey_stops.append(tk.StringVar(journey_stop_frame, value=''))
    tk.Label(journey_stop_frame, text='Box Name').pack(side='left')
    tk.Entry(journey_stop_frame, textvariable=journey_stops[-1]).pack(side='left')
    journey_stop_frame.pack()


# Parameters Function
def add_parameters():
    parameters_window = tk.Toplevel(root)
    parameters_window.title('Parameters')
    
    # Add parameter fields
    tk.Label(parameters_window, text='Median survey time for BH boxes (minutes)').grid(row=0, column=0)
    tk.Entry(parameters_window, textvariable=median_survey_time_BH).grid(row=0, column=1)
    tk.Label(parameters_window, text='Median survey time for CPT boxes (minutes)').grid(row=1, column=0)
    tk.Entry(parameters_window, textvariable=median_survey_time_CPT).grid(row=1, column=1)
    tk.Label(parameters_window, text='Fixed time (minutes)').grid(row=2, column=0)
    tk.Entry(parameters_window, textvariable=fixed_time).grid(row=2, column=1)
    tk.Label(parameters_window, text='Speed in knots').grid(row=3, column=0)
    tk.Entry(parameters_window, textvariable=speed_in_knots).grid(row=3, column=1)
    tk.Label(parameters_window, text='Minimum distance (kilometers)').grid(row=4, column=0)
    tk.Entry(parameters_window, textvariable=min_distance).grid(row=4, column=1)

# Run Prediction Function
def run_prediction():
    boxes = [stop.get() for stop in journey_stops]
    variable_time = 60 / (float(speed_in_knots.get()) * 1.852)  # convert from knots to km/hr and then to min/km
    journey_time, travel_times, end_time, journey_summary = predict_journey_time(
        boxes, float(median_survey_time_BH.get()), float(median_survey_time_CPT.get()), 
        float(fixed_time.get()), variable_time, float(min_distance.get()), box_locations, start_time.get()
    )

    # Print results
    result_text = f"Estimated Total Journey Time for the given list of boxes: {journey_time} minutes or approximately {journey_time/60:.2f} hours.\n"
    result_text += "Estimated Travel Times between consecutive boxes: " + ', '.join(str(t) for t in travel_times) + "\n"
    result_text += f"Estimated End Time for the journey: {end_time.strftime('%H:%M:%S')}\n"
    result_text += "\nJourney Summary:"
    for i, details in enumerate(journey_summary):
        result_text += f"\nBox {i+1}: {details['box_name']}\n"
        result_text += f"Start Time: {details['start_time'].strftime('%H:%M:%S')}\n"
        result_text += f"End Time: {details['end_time'].strftime('%H:%M:%S')}\n"
        result_text += f"Time Spent on Box: {details['box_time']} minutes\n"
        if 'travel_time_to_next_box' in details:
            result_text += f"Travel Time to Next Box: {details['travel_time_to_next_box']} minutes\n"
    result_label['text'] = result_text
    save_button.pack()

def save_excel():
    # Convert summary to DataFrame
    summary_df = pd.DataFrame(journey_summary)
    # Convert datetime columns to strings for better readability in Excel
    summary_df['start_time'] = summary_df['start_time'].apply(lambda x: x.strftime('%H:%M:%S'))
    summary_df['end_time'] = summary_df['end_time'].apply(lambda x: x.strftime('%H:%M:%S'))
    # Ask for save location and write to Excel
    filename = filedialog.asksaveasfilename(defaultextension=".xlsx")
    summary_df.to_excel(filename, index=False)

root = tk.Tk()

file_button = tk.Button(root, text="Select Survey Log", command=load_database)
file_button.pack()

journey_stops = []
add_box_button = tk.Button(root, text="Add Box", command=add_journey_stop)
add_box_button.pack()

start_time_label = tk.Label(root, text="Start Time (HH:MM:SS)")
start_time_label.pack()
start_time = tk.StringVar(root, value='HH:MM:SS')
start_time_entry = tk.Entry(root, textvariable=start_time)
start_time_entry.pack()

params_button = tk.Button(root, text="Parameters", command=add_parameters)
params_button.pack()

run_button = tk.Button(root, text="Estimate ETA", command=run_prediction)
run_button.pack()

result_label = tk.Label(root, text="")
result_label.pack()

save_button = tk.Button(root, text="Export to Excel", command=save_excel)
# save_button will be packed after running the prediction

# Initialize parameters
median_survey_time_BH = tk.StringVar(root, value='170')
median_survey_time_CPT = tk.StringVar(root, value='100')
fixed_time = tk.StringVar(root, value='8.77')
speed_in_knots = tk.StringVar(root, value='9.46')
min_distance = tk.StringVar(root, value='0.5')

root.mainloop()
